In [126]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from scipy.stats import t


dir = "C:/Users/foresight_User/Desktop/公司文件/4.CIT季賽/測試資料/"
# CH_TO = pd.read_csv(dir+"CH_TO.csv")
# X = CH_TO.drop(columns=["Y","Context Name"])#.to_numpy()
# y = CH_TO["Y"]#.to_numpy()
# #加入CROSS項目，有時候會顯著，有時候不會顯著，明顯受到切資料的影響， 但CROSS BEST的模型確實會比較好


#
X = pd.read_csv(dir+"wb/Indicator_Data(1).csv").drop(columns=["Unnamed: 74","Context Name"])#.to_numpy()
y = pd.read_csv(dir+"wb/Metrology_Data(1).csv")["Point1"]#.to_numpy()
#


data = pd.read_excel(dir+"Array7_N808測試資料集/Model_TJN808XK_SAMP75.xlsx").drop(columns=["CONTEXTID"])
X = data.drop(columns=list(filter(lambda x: "T2_CD" in x,data.columns)))#.to_numpy()
X = X.fillna(value=0)
y = data["T2_CD01_Mean"]#.to_numpy()




#X = torch.FloatTensor(X).requires_grad_(True)
#y = torch.FloatTensor(y).requires_grad_(True)

X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.3,random_state=np.random.randint(0,100))

In [127]:
import sklearn
sklearn.__version__

'1.3.2'

In [128]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor()
reg.fit(X_train,y_train)
y_pred = reg.predict(X_test)

In [129]:
r2_score(y_test,y_pred)

-0.23910509084044484

In [130]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error,mean_absolute_percentage_error
# 計算 Deviance
from sklearn.metrics import mean_tweedie_deviance
from sklearn.preprocessing import MinMaxScaler



def deviance(real, predicted):
    if isinstance(predicted, pd.Series):
        predicted = predicted.to_numpy()
    real = real.to_numpy().reshape(-1)
    predicted =  predicted.reshape(-1)
    scaler =  MinMaxScaler().fit(np.concatenate((real, predicted),axis=0).reshape(-1,1))
    real_std = scaler.transform(real.reshape(-1, 1))
    predicted_std = scaler.transform(predicted.reshape(-1, 1))
    if sum(predicted_std < 0) or sum(real_std < 0):
        nag_idx = np.concatenate([np.where(predicted_std < 0)[0], np.where(real_std < 0)[0]])
        predicted_std = np.delete(predicted_std, nag_idx, 0)
        real_std = np.delete(real_std, nag_idx, 0)
    return mean_tweedie_deviance(real_std, predicted_std)

# 計算 RMSLE (Root Mean Squared Logarithmic Error)
def root_mean_squared_log_error(real, predicted):
    if isinstance(predicted, pd.Series):
        predicted = predicted.to_numpy()
    real = real.to_numpy().reshape(-1)
    predicted =  predicted.reshape(-1)
    scaler =  MinMaxScaler().fit(np.concatenate((real, predicted),axis=0).reshape(-1,1))
    real_std = scaler.transform(real.reshape(-1, 1))
    predicted_std = scaler.transform(predicted.reshape(-1, 1))
    if sum(predicted_std < 0) or sum(real_std < 0):
        nag_idx = np.concatenate([np.where(predicted_std < 0)[0], np.where(real_std < 0)[0]])
        predicted_std = np.delete(predicted_std, nag_idx, 0)
        real_std = np.delete(real_std, nag_idx, 0)
    return mean_squared_log_error(real_std, predicted_std, squared=False)


def _95_error(real, predicted):
    if isinstance(predicted, pd.Series):
        predicted = predicted.to_numpy()
    real = real.to_numpy().reshape(-1)
    predicted = predicted.reshape(-1)
    residual = abs(real - predicted)
    sorted_residual = sorted(residual)
    ninety_five_err = sorted_residual[: int(len(sorted_residual) * 0.95)][-1]
    return ninety_five_err
def adjusted_r2_score(real, predicted, n, p):
    r2 = r2_score(real, predicted)
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))
    return adjusted_r2



In [131]:

from typing import Union
import numpy as np
import pandas as pd
from scipy.stats import t

def confidence_interval(data, confidence):
    n = len(data)
    mean_value = np.mean(data)
    std_error = np.std(data, ddof=1) / np.sqrt(n)
    margin_error = std_error * t.ppf((1 + confidence) / 2., n - 1)
    return mean_value, mean_value - margin_error, mean_value + margin_error

def monte_carlo_simulation(y_true, n_simulations):
    y_preds_random = []
    for _ in range(n_simulations):
        noise = np.random.normal(0, np.std(y_true) * 1, size=len(y_true))
        y_pred_random = y_true + noise
        y_preds_random.append(y_pred_random)
    return np.array(y_preds_random)
def random_guess(metric_fuc, y_true, y_preds_random):
    values = [metric_fuc(y_true, y_pred_random) for y_pred_random in y_preds_random]
    return values

def evaluation_metric(metric_fuc,y_true,y_pred):
    return metric_fuc(y_true, y_pred)

def generate_evaluation_dataframe(y_true: Union[pd.Series, np.ndarray] = None, y_pred: Union[pd.Series, np.ndarray] = None, n_simulations:int =100, confidence:float = 0.95, metrics_info:dict ={},direction:str=None):
    if not isinstance(y_true, (pd.Series, np.ndarray)):
        raise TypeError("Only pd.Series and np.ndarray are allowed")

    y_preds_random = monte_carlo_simulation(y_true, n_simulations)
    # 構建DataFrame
    data = []
    for metric_name,metric_fuc in metrics_info.items():
        
        values = random_guess(metric_fuc,y_true, y_preds_random)
        _,lower_bound, upper_bound = confidence_interval(values, confidence)
        score = evaluation_metric(metric_fuc,y_true,y_pred)
        if direction == "maximum":
            if score > upper_bound:
                status = 'Fair' #
            elif (score <= upper_bound) and (score >= lower_bound):
                status = 'Poor' #
            elif score < lower_bound:
                status = 'Extremely Poor' #
            else:
                status = np.nan  # 預防未定義的情況
        elif direction == "minimum":
            if score < lower_bound:
                status = 'Fair' #
            elif (score <= upper_bound) and (score >= lower_bound):
                status = 'Poor' #
            elif score > upper_bound:
                status = 'Extremely Poor' #
            else:
                status = np.nan  # 預防未定義的情況
        data.append([score, confidence, metric_name, lower_bound, upper_bound, status])
    
    df = pd.DataFrame(data, columns=['score', 'confidence', 'metrics', 'rando_guess_lower_bound', 'rando_guess_upper_bound', 'status'])
    return df



# 範例一
metric_minimum ={'MAE':mean_absolute_error, 'MAPE':mean_absolute_percentage_error, 'MSE':mean_squared_error, 'Deviance':deviance, 'RMSLE':root_mean_squared_log_error}# 這邊放哪個評估分數越小越好

generate_evaluation_dataframe(y_true=y_test, y_pred=y_pred,metrics_info=metric_minimum,direction = "minimum")#


# 範例二
metric_maximum ={'R2':r2_score} # 這邊放哪個評估分數越大越號
generate_evaluation_dataframe(y_true=y_test, y_pred=y_pred,metrics_info=metric_maximum,direction = "maximum")

,score,confidence,metrics,rando_guess_lower_bound,rando_guess_upper_bound,status
0,-0.239105,0.95,R2,-0.017808,0.09933,Extremely Poor


In [132]:
metric_minimum ={'MAE':mean_absolute_error, 'MAPE':mean_absolute_percentage_error, 'MSE':mean_squared_error, 'Deviance':deviance, 'RMSLE':root_mean_squared_log_error}# 這邊放哪個評估分數越小越好

generate_evaluation_dataframe(y_true=y_test, y_pred=y_pred,metrics_info=metric_minimum,direction = "minimum")#



,score,confidence,metrics,rando_guess_lower_bound,rando_guess_upper_bound,status
0,0.121376,0.95,MAE,0.098982,0.104923,Extremely Poor
1,0.028918,0.95,MAPE,0.023560,0.024978,Extremely Poor
2,0.020507,0.95,MSE,0.015430,0.017303,Extremely Poor
3,0.082140,0.95,Deviance,0.032707,0.037790,Extremely Poor
4,0.204425,0.95,RMSLE,0.126507,0.135704,Extremely Poor


In [133]:
metric_maximum ={'R2':r2_score} # 這邊放哪個評估分數越大越號
generate_evaluation_dataframe(y_true=y_test, y_pred=y_pred,metrics_info=metric_maximum,direction = "maximum")

,score,confidence,metrics,rando_guess_lower_bound,rando_guess_upper_bound,status
0,-0.239105,0.95,R2,-0.064532,0.064059,Extremely Poor


In [139]:
generate_evaluation_dataframe(y_true=y_test.to_list(), y_pred=y_pred,metrics_info=metric_minimum,direction = "minimum")#


TypeError: Only pd.Series and np.ndarray are allowed

In [138]:
y_pred

array([4.19401993, 4.23459339, 4.23224582, 4.27229757, 4.24244304,
       4.20158293, 4.3247526 , 4.19568755, 4.21431532, 4.24016197,
       4.24225062, 4.24905322, 4.15974196, 4.24481582, 4.2317389 ,
       4.26326687, 4.23491014, 4.2474046 , 4.25394009, 4.17030641,
       4.18797309, 4.28438436, 4.25293465])

In [353]:
# def generate_evaluation_dataframe(y_true, y_pred, n_simulations:int =100, confidence:list =[0.95, 0.997], metrics_info:dict ={}):
#     y_preds_random = monte_carlo_simulation(y_true, n_simulations)
#     # 構建DataFrame
#     data = []
#     for metric_name,metric_fuc in metrics_info.items():
        
#         values = random_guess(metric_fuc,y_true, y_preds_random)
#         for conf in confidence:
#             _,lower_bound, upper_bound = confidence_interval(values, conf)
#             score = evaluation_metric(metric_fuc,y_true,y_pred)
#             if score < lower_bound:
#                 status = 'below_CI'
#             elif score > upper_bound:
#                 status = 'above_CI'
#             else:
#                 status = 'within_CI'
#             data.append([score, conf, metric_name, lower_bound, upper_bound, status])
    
#     return pd.DataFrame(data, columns=['score', 'confidence', 'metrics', 'lower_bound', 'upper_bound', 'status'])

# def generate_evaluation_dataframe(y_true, y_pred, n_simulations:int =100, confidence:list =[0.95, 0.997], metrics_info:dict ={},direction:str=None,transform=None):
#     y_preds_random = monte_carlo_simulation(y_true, n_simulations)
#     # 構建DataFrame
#     data = []
#     for metric_name,metric_fuc in metrics_info.items():
        
#         values = random_guess(metric_fuc,y_true, y_preds_random)
#         for conf in confidence:
#             _,lower_bound, upper_bound = confidence_interval(values, conf)
#             score = evaluation_metric(metric_fuc,y_true,y_pred)
#             if score < lower_bound:
#                 status = 'below_CI'
#             elif score > upper_bound:
#                 status = 'above_CI'
#             else:
#                 status = 'within_CI'
#             data.append([score, conf, metric_name, lower_bound, upper_bound, status])
#     result = pd.DataFrame(data, columns=['score', 'confidence', 'metrics', 'lower_bound', 'upper_bound', 'status'])

#     if transform !=True:
#         return result
#     else:
#         data = []
#         for metric in metrics_info.keys():
#             df = result[result["metrics"] == metric]
#             if direction == "minimum":
#                 if ((df.confidence == 0.997) & (df.status == "below_CI")).all():
#                     state = 'Good'
#                 elif ((df.confidence == 0.997) &  (df.status == "within_CI")).all() and ((df.confidence == 0.95) & (df.status == "below_CI")).all():
#                     state = 'Fair'
#                 elif ((df.confidence == 0.997) & (df.status == "within_CI")).all() and ((df.confidence == 0.95) & (df.status == "within_CI")).all():
#                     state = 'Poor'
#                 elif ((df.confidence == 0.997) & df.status == "within_CI").all() and ((df.confidence == 0.95) & (df.status == "above_CI")).all():
#                     state = 'Very Poor'
#                 elif ((df.confidence == 0.997) & (df.status == "above_CI")).all():
#                     state = 'Extremely Poor'
#                 else:
#                     state = np.nan
#             elif direction == "maximum":
#                 if ((df.confidence == 0.997) & (df.status == "above_CI")).all():
#                     state = 'Good'
#                 elif ((df.confidence == 0.997) and (df.status == "within_CI")).all() and ((df.confidence == 0.95) and (df.status == "above_CI")).all():
#                     state = 'Fair'
#                 elif ((df.confidence == 0.997) and (df.status == "within_CI")).all() and ((df.confidence == 0.95) and (df.status == "within_CI")).all():
#                     state = 'Poor'
#                 elif ((df.confidence == 0.997) and (df.status == "within_CI")).all() and (df.confidence == 0.95 and (df.status == "below_CI")).all():
#                     state = 'Very Poor'
#                 elif ((df.confidence == 0.997) and (df.status == "below_CI")).all():
#                     state = 'Extremely Poor'
#                 else:
#                     state = np.nan
#             data.append([metric,state])
#         result = pd.DataFrame(data, columns=[ 'metrics','state'])
#         return result

In [120]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb
from optuna.pruners import ThresholdPruner,PercentilePruner,SuccessiveHalvingPruner
import optuna
from sklearn.datasets import make_regression


random_state = 3
n_train_iter = 1
X, y = make_regression(n_samples=100000,n_features=40,n_informative=40, noise=0.5,random_state=random_state)
# X, y = load_iris(return_X_y=True)
X_valid, _, y_valid, _ = train_test_split(X,y)
# classes = np.unique(y)


def objective(trial):
    
    auto_tuning_params = {
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.1, 1.0),
        'gamma': trial.suggest_float("gamma", 0.0, 9.0),
        'learning_rate': trial.suggest_float("learning_rate", 1e-6, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 18, 1),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 5),
        'n_estimators': trial.suggest_int('n_estimators', 200, 8000, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 2.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        'random_state': random_state
    }
    clf = xgb.XGBRegressor(**auto_tuning_params)

    clf.fit(X_valid, y_valid)
    score = clf.score(X_valid, y_valid)
    return score

In [121]:
# tolerence = 1,2,3,4,5,6,7,8,9,10 .....
n_trials = 10
tolerence = 1
if tolerence > 1:
    # n_trials = int(n_trials/tolerence**(2/5))
    # if n_trials == 0:
    #     n_trials = 1
    study = optuna.create_study(
        direction="maximize", pruner=SuccessiveHalvingPruner(reduction_factor=tolerence)
    )
else:
    study = optuna.create_study(
        direction="maximize"
    )    


study.optimize(objective, n_trials=n_trials)

[I 2024-06-21 08:23:55,020] A new study created in memory with name: no-name-4d8a8300-a246-472e-bc58-4554333e83a2
C:\Users\foresight_User\AppData\Local\Temp\ipykernel_2500\1708608048.py:25: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 3, 18, 1),
C:\Users\foresight_User\AppData\Local\Temp\ipykernel_2500\1708608048.py:27: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'n_estimators': trial.suggest_int('n_estimators', 200, 8000, 100),
[I 2024-06-21 08:27:25,157] Trial 0 finished with value: -1.0738938765298764e+24 and parameters: {'colsample_bytree': 0.5990505973530461, 'gamma': 0.7038913828895049, 'learning_rate': 0.6846363438204282, 'max_depth': 11, 'min_child_weight': 4, 'n_estimators': 5800, 'reg_alpha': 1.4855655100548075, 'reg_lambda': 0.837369033452857, 'subsample': 0.255477046160732

In [123]:
# tolerence = 1,2,3,4,5,6,7,8,9,10 .....
n_trials = 10
tolerence = 10 # 1 速度, 3 平衡, 7 準確
if tolerence > 1:
    n_trials = int(n_trials/tolerence**(2/5))
    if n_trials == 0:
        n_trials = 1
    study = optuna.create_study(
        direction="maximize", pruner=SuccessiveHalvingPruner(reduction_factor=tolerence)
    )
else:
    study = optuna.create_study(
        direction="maximize"
    )    
study.optimize(objective, n_trials=n_trials,n_jobs=-1)

[I 2024-06-21 08:52:01,177] A new study created in memory with name: no-name-29562e06-1ce5-43ab-ac39-52cf6f08046b
C:\Users\foresight_User\AppData\Local\Temp\ipykernel_2500\1708608048.py:25: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 3, 18, 1),
C:\Users\foresight_User\AppData\Local\Temp\ipykernel_2500\1708608048.py:27: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'n_estimators': trial.suggest_int('n_estimators', 200, 8000, 100),
[I 2024-06-21 08:52:54,370] Trial 0 finished with value: 0.9999758879990331 and parameters: {'colsample_bytree': 0.28650608188464566, 'gamma': 4.188009165882256, 'learning_rate': 0.0836222192698602, 'max_depth': 8, 'min_child_weight': 4, 'n_estimators': 3200, 'reg_alpha': 1.415757023380406, 'reg_lambda': 0.10982493369013135, 'subsample': 0.5461107455861622}. B

In [110]:
n_trials

3

In [69]:
def objective(trial):
    
    auto_tuning_params = {
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.1, 1.0),
        'gamma': trial.suggest_float("gamma", 0.0, 9.0),
        'learning_rate': trial.suggest_float("learning_rate", 1e-6, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 18, 1),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 5),
        'n_estimators': trial.suggest_int('n_estimators', 200, 8000, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 2.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        'random_state': random_state
    }
    clf = xgb.XGBRegressor(**auto_tuning_params)

    clf.fit(X_valid, y_valid)
    score = clf.score(X_valid, y_valid)
    return score


study = optuna.create_study(
    direction="maximize", pruner=PercentilePruner(percentile=0.25,interval_steps =10)
)
study.optimize(objective, n_trials=5)

[I 2024-06-19 16:41:08,104] A new study created in memory with name: no-name-f3d034cc-5d59-432e-bcba-56abb032dc16
C:\Users\foresight_User\AppData\Local\Temp\ipykernel_2500\3674623702.py:7: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 3, 18, 1),
C:\Users\foresight_User\AppData\Local\Temp\ipykernel_2500\3674623702.py:9: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'n_estimators': trial.suggest_int('n_estimators', 200, 8000, 100),
[I 2024-06-19 16:41:44,177] Trial 0 finished with value: 0.9999982904523037 and parameters: {'colsample_bytree': 0.4571502946746342, 'gamma': 1.6200436964351117, 'learning_rate': 0.39821741104297426, 'max_depth': 7, 'min_child_weight': 2, 'n_estimators': 5200, 'reg_alpha': 0.7415140738282958, 'reg_lambda': 0.12049155386041477, 'subsample': 0.4892590913398624}. B

KeyboardInterrupt: 